# G-Research Crypto Forecasting

G-ReSearch Crypto Forecasting is a time series data, which contains daily price of cryptocurrencies. Our task is to  forecast short term returns in 14 popular cryptocurrencies.

The purpose of this notebook is to share basic approach to time series data analysis. 

## Contents
1. [Basic EDA](#eda)
2. [Time series feature engineering](#fe)
3. [Modeling](#model)
 + [Time series cross validation](#tscv)
 + [Hyperparameter tuning with Optuna](#optuna)

## Load packages and data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf, arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.stats.stats import pearsonr

import time
from datetime import datetime
from tqdm import tqdm

In [2]:
train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
sup_train = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')

print(train.shape)
print(sup_train.shape)

(24236806, 10)
(2518278, 10)


## 1. Basic EDA <a class="anchor" id="eda"></a>
This part contains simple exploration of G-Research data. Before we get into EDA, we need to change format of timestamp feature and create some new features.

In [3]:
# change format of timestamp variable
train['timestamp'] = pd.to_datetime(train['timestamp'], unit = 's')

# Diff: difference between close and open price of crypto
train['Diff'] = train['Close'] - train['Open']

In [4]:
# order by asset_id (ascending)
asset_details = asset_details.sort_values(by = 'Asset_ID').reset_index(drop = True)
asset_details

,Asset_ID,Weight,Asset_Name
0,0,4.304065,Binance Coin
1,1,6.779922,Bitcoin
2,2,2.397895,Bitcoin Cash
3,3,4.406719,Cardano
4,4,3.555348,Dogecoin
5,5,1.386294,EOS.IO
6,6,5.894403,Ethereum
7,7,2.079442,Ethereum Classic
8,8,1.098612,IOTA
9,9,2.397895,Litecoin


### Comparison of average trading volume of assets
This graph shows average trading volume of each asset. **Bitcoin, Dodgecoin and Ethereum** are 3 most frequently traded assets among 14 cryptos.

In [5]:
assets = train.groupby('Asset_ID')['Count'].mean().reset_index()

colors = ['lightgrey']*14
colors[1] = colors[4] = colors[6] = '#3366ff'
assets_bar = go.Bar(x = assets['Asset_ID'], y = assets['Count'], marker_color = colors)
data = [assets_bar]
layout = go.Layout(title = 'Average trading volume of each asset')
fig = go.Figure(data = data, layout = layout)

fig.update_traces(marker_line_width = 1,marker_line_color = "black")
fig.update_layout(
    title = {
        'text': 'Average trading volume of each asset',
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} , 
    
    xaxis = dict(
        tickvals = list(range(0, 14)),
        ticktext = asset_details.Asset_Name
    ),
    template = "plotly_white")

fig.update_xaxes(title_text = 'Asset')
fig.update_yaxes(title_text = 'Trading volume')

fig

### Candlestick chart of recent 1 year asset price
This is candlestick chart of recent 1 year price of specific asset. I plotted chart of 3 most frequently traded assets, Bitcoin, Dogecoin and Ethereum. Overall trend of price of those 3 assets are quite similar.

Reference: <https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition/notebook#Building-your-prediction-model>

In [6]:
def candelstick_chart(data, id_, title):
    data = data[data['Asset_ID'] == id_].reset_index(drop = True)
    data = data.set_index('timestamp')
    data = data.iloc[-365:,:]  # recent 1 year
    data['MA5'] = data['Close'].rolling(window = 5, min_periods = 1).mean()
    data['MA30'] = data['Close'].rolling(window = 30, min_periods = 1).mean()
    data['MA120'] = data['Close'].rolling(window = 120, min_periods = 1).mean()
    
    candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    candlestick.update_xaxes(title_text = 'Time')

    candlestick.update_layout(
    title = {
        'text': '{:} Candelstick Chart'.format(title),
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} , 
    template="plotly_white")

    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

In [7]:
candelstick_chart(train, 1, 'Bitcoin')

In [8]:
candelstick_chart(train, 4, 'Dodgecoin')

In [9]:
candelstick_chart(train, 6, 'Ethereum')

### Check overall trend: Moving Average
To check overall trend of asset price, we use **moving average(MA)** in general. In finance, a moving average (MA) is a stock indicator that is commonly used in technical analysis. The reason for calculating the moving average of a stock is to help smooth out the price data by creating a constantly updated average price.

 + Simple moving average(SMA): calculation that takes the arithmetic mean of a given set of prices over the specific number of days in the past
 + Exponential moving average(EMA): weighted average that gives greater importance to the price of a stock in more recent days, making it an indicator that is more responsive to new information.
 
 
In this part, I'll calculate SMA of close price of asset for 5(weekly), 30(monthly) and 120 days. MA5 and MA30 shows short-term trend and MA120 shows long-term trend of asset price.

In [10]:
def ma_chart(data, id_, title):
    data = data[data['Asset_ID'] == id_].reset_index(drop = True)
    data = data.set_index('timestamp')
    data = data.iloc[-365:,:]  # recent 1 year
    data['MA5'] = data['Close'].rolling(window = 5, min_periods = 1).mean()
    data['MA30'] = data['Close'].rolling(window = 30, min_periods = 1).mean()
    data['MA120'] = data['Close'].rolling(window = 120, min_periods = 1).mean()
    
    ma = go.Figure(data = [go.Scatter(x = data.index, y = data['Close'], mode='lines', 
                                     name = 'Close', line = dict(color = 'black', width = 2))])
    ma.update_xaxes(title_text = 'Time')

    ma.update_layout(
    title = {
        'text': '{:} Moving Average'.format(title),
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} , 
    template="plotly_white")
    
    ma.add_trace(go.Scatter(x = data.index, y = data['MA5'], mode='lines', 
                                     name='MA5', line = dict(color = 'red', width = 2)))
    ma.add_trace(go.Scatter(x = data.index, y = data['MA30'], mode='lines', 
                                     name='MA30', line = dict(color = 'blue', width = 2)))
    ma.add_trace(go.Scatter(x = data.index, y = data['MA120'], mode='lines', 
                                     name='MA120', line = dict(color = 'orange', width = 2)))

    ma.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return ma

In [11]:
# Moving average chart of Bitcoin
ma_chart(train, 1, 'Bitcoin')

### Missing values
There are some missing values in **Target** variable and **VWAP**. All missing values in VWAP variable belong to Asset 10, Maker. Since there are only 9 missings in this variable, we'll just remove those data. Missing in Target will not be included in modeling.

In [12]:
# Target - missing 3% 
train.isna().sum().sort_values(ascending = False)

Target       750338
VWAP              9
timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
Diff              0
dtype: int64

In [13]:
# check missing in VWAP
train[train['VWAP'].isna()]

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,Diff
15004269,2020-06-14 22:26:00,10,2.0,501.00,501.00,501.00,501.00,0.0,NaN,NaN,0.0
15004283,2020-06-14 22:27:00,10,4.0,501.00,501.00,501.00,501.00,0.0,NaN,NaN,0.0
15059232,2020-06-17 20:36:00,10,2.0,529.77,529.77,529.77,529.77,0.0,NaN,0.007356,0.0
15143187,2020-06-22 11:02:00,10,2.0,503.60,503.60,503.60,503.60,0.0,NaN,-0.005672,0.0
15183088,2020-06-24 14:29:00,10,2.0,484.16,484.16,484.16,484.16,0.0,NaN,-0.004281,0.0
15184216,2020-06-24 15:52:00,10,2.0,480.00,480.00,480.00,480.00,0.0,NaN,NaN,0.0
15184243,2020-06-24 15:54:00,10,2.0,480.00,480.00,480.00,480.00,0.0,NaN,NaN,0.0
15184309,2020-06-24 15:59:00,10,6.0,479.07,479.07,479.07,479.07,0.0,NaN,NaN,0.0
15184778,2020-06-24 16:34:00,10,4.0,478.00,478.00,475.00,478.00,0.0,NaN,NaN,0.0


In [14]:
# remove NA values in VWAP
train = train[train['VWAP'].isna() == False]

## 2. Time series Feature Engineering <a class="anchor" id="fe"></a>
We will use asset Bitcoin data only for feature engineering and modeling in this notebook. For other 13 assets, you can repeat same process.

In [15]:
# Bitcoin only
data = train[train['Asset_ID'] == 1]

In [16]:
# remove rows without target 
data = data[data['Target'].isna() == False]
data

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,Diff
2,2018-01-01 00:01:00,1,229.0,13835.19400,14013.80,13666.11,13850.176000,31.550062,13827.062093,-0.014643,14.982000
10,2018-01-01 00:02:00,1,235.0,13835.03600,14052.30,13680.00,13828.102000,31.046432,13840.362591,-0.015037,-6.934000
18,2018-01-01 00:03:00,1,528.0,13823.90000,14000.40,13601.00,13801.314000,55.061820,13806.068014,-0.010309,-22.586000
26,2018-01-01 00:04:00,1,435.0,13802.51200,13999.00,13576.28,13768.040000,38.780529,13783.598101,-0.008999,-34.472000
34,2018-01-01 00:05:00,1,742.0,13766.00000,13955.90,13554.44,13724.914000,108.501637,13735.586842,-0.008079,-41.086000
...,...,...,...,...,...,...,...,...,...,...,...
24236515,2021-09-20 23:40:00,1,2643.0,42632.46500,42736.90,42607.50,42703.636250,100.797218,42653.031385,0.002084,71.171250
24236529,2021-09-20 23:41:00,1,2281.0,42718.81500,42819.38,42690.84,42781.970571,76.339988,42755.785162,0.003246,63.155571
24236543,2021-09-20 23:42:00,1,2642.0,42772.92125,42827.10,42690.75,42755.592500,117.429123,42749.075916,0.003108,-17.328750
24236557,2021-09-20 23:43:00,1,2134.0,42762.29000,42811.30,42694.37,42717.234286,78.049458,42749.024591,0.002770,-45.055714


### Additional features
I created several additional features for modeling, including **time related features**. In time series analysis, creating time related features can significantly increase model performance. I used 2 mostly used technique for time series analysis: **moving average** and **lagged features**.

1. Moving average 
 + Above explanation with graph
<br>
2. Lagged feature
 + In time series analysis, future value is greatly affected by past values. Those past values are **lags** and we use thos lag features to enhance model performance.

In [17]:
# df should be particular asset
def get_feats(df):
    df['upper_shadow'] = df['High'] - np.maximum(df['Close'], df['Open'])
    df['lower_shadow'] = np.minimum(df['Close'], df['Open']) - df['Low']
    
    # average trading volume
    df['avg_volume'] = df['Volume'] / df['Count']
    
    # average price
    df['avg_price'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    
    # difference between open and close price
    df.rename(columns = {'Diff': 'diff_open_close'})
    
    # moving average features - short term and long term
    df['ma_20'] = df['Close'].rolling(window = 20, min_periods = 1).mean()
    df['ma_120'] = df['Close'].rolling(window = 120, min_periods = 1).mean()
    
    # lagged features
    lags_ = [5, 20, 60, 120]
    for lag in lags_:
        df['lag_' + str(lag)] = df['Target'].shift(lag)
        
    df = df.fillna(0)
    
    return df

In [18]:
# lagged features
data = get_feats(data)

#### FE function
Below function creates time-series features of each assets. You can just enter asset_id in **asset_feats** function to get features for that particular asset.

In [19]:
# feature engineering for other assets 
def asset_feats(df, asset_id):
    data = df[df['Asset_ID'] == asset_id]
    data = data[data['Target'].isna() == False]
    data = get_feats(data)
    return data

## 3. Modeling <a class="anchor" id="model"></a>

### (1) Time Series Cross Validation <a class="anchor" id="tscv"></a>
For time series modeling, I will use boosting regressors, LGBMRegressor and XGBRegressor, which show great performance in general. First step is to compare performance of two models with time series cross validation. In time series analysis, it's not recommended to apply KFold or Stratified KFold since observations in past influences current values. Instead, we use **Time series split** in this case. Below image shows how data is splitted if you apply time series split. Observations from the training set occur before their corresponding validation set. We will use 10-fold time series cross validation in this notebook. (Evaluation metric is Pearson Correlation Coefficient)
<br>
![tcsv](https://miro.medium.com/max/1204/1*qvdnPF8ETV9mFdMT0Y_BBA.png)

In [20]:
# create X and y(target)
X = data.drop(['timestamp', 'Asset_ID'], axis = 1)
y = data['Target']

In [21]:
# 10-fold time series cross validation
def timecv_model(model, X, y):
    tfold = TimeSeriesSplit(n_splits = 10)
    pcc_list = []
    for _, (train_index, test_index) in tqdm(enumerate(tfold.split(X), start=1)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        pcc = pearsonr(pred, y_test) 
        pcc_list.append(pcc[0])
    
    return pcc_list

def cv_result(model, X, y):
    model_name = model.__class__.__name__
    pcc_ = timecv_model(model, X, y)
    for i, pcc in enumerate(pcc_):
        print(f'{i}th fold: {model_name} PCC: {pcc:.4f}')
    print(f'\n{model_name} average PCC: {np.mean(pcc_):.4f}')

Below are results of 10-fold time series cross validation(TSCV) of LGBMRegressor and XGBRegressor. It takes much longer time in XGBRegressor. Parameters are randomly selected, just to compare general performance of two models. We will choose better model with TSCV and then conduct hyperparameter tuning.

In [22]:
lgb_model = LGBMRegressor(n_estimators = 1500,
                      max_depth = 10,
                      num_leaves = 20,
                      colsample_bytree = 0.8,
                      subsample = 0.7,
                      seed = 0)

cv_result(lgb_model, X, y)

10it [09:00, 54.04s/it]

0th fold: LGBMRegressor PCC: 0.9919
1th fold: LGBMRegressor PCC: 0.9863
2th fold: LGBMRegressor PCC: 0.9869
3th fold: LGBMRegressor PCC: 0.9707
4th fold: LGBMRegressor PCC: 0.8944
5th fold: LGBMRegressor PCC: 0.9707
6th fold: LGBMRegressor PCC: 0.9774
7th fold: LGBMRegressor PCC: 0.9755
8th fold: LGBMRegressor PCC: 0.9359
9th fold: LGBMRegressor PCC: 0.9804

LGBMRegressor average PCC: 0.9670


In [23]:
#xgb_model = XGBRegressor(n_estimators = 1500,
                         #max_depth = 10,
                         #min_child_weight = 5,
                         #gamma = 0.1)

#cv_result(xgb_model, X, y)

### (2) Optuna
Now, tune parameters to get optimal parameter for LGBMRegressor. There are many hyperparameter tuning techinques: GridSearchCV, RandomSearchCV, Bayesian Optimization..etc..

But I will try **Optuna** for hyperparameter tuning. Optuna is widely used in many data analysis platforms these days to get optimal parameters for model. Information about optuna is well explained [here](https://medium.com/@kalyaniavhale7/understanding-of-optuna-a-machine-learning-hyperparameter-optimization-framework-ed31ebb335b9). (I summarized explanations about optuna referring to this link.)

In [24]:
data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d')

print('Earliest date: ', min(data['timestamp']))
print('Lastest date: ', max(data['timestamp']))

Earliest date:  2018-01-01
Lastest date:  2021-09-20


Bitcoin data starts from 2018-01-01 to 2021-09-20. I diveded full dataset into train and valid dataset first: Train data contains data before 2020-09-20 and validation data contains data after 2020-09-20. The ratio of train and valid size is apporixmately 73:27.

In [25]:
train = data[data['timestamp'] < '2020-09-20']
valid = data[data['timestamp'] >= '2020-09-20']

train.drop(['timestamp', 'Asset_ID'], axis = 1, inplace = True)
valid.drop(['timestamp', 'Asset_ID'], axis = 1, inplace = True)

X_train = train.drop(['Target'], axis = 1)
y_train = train['Target']
X_valid = valid.drop(['Target'], axis = 1)
y_valid = valid['Target']

print(X_train.shape)
print(X_valid.shape)

(1428968, 18)
(527010, 18)


### Objective function <a class="anchor" id="optuna"></a>
Our objective of hyperparameter tuning is to find parameter that maximizes or minimized output of objective function. If evaluation metric is logloss, we have to minimized objective function but if it is accuracy, we have to maximize objective function. During the optimization, Optuna repeatedly calls and evaluates the objective function with different parameters.

In objective function, we define parameter search space.
 + categorical parameter: optuna.trial.Trial.suggest_categorical()
 + integer parameter: optuna.trial.Trial.suggest_int()
 + float parameter: optuna.trial.Trial.suggest_float()
 
 
 
In Optuna, we use the study object to manage optimization. Method create_study() returns a study object. A study object has useful properties for analyzing the optimization outcome. 
 + create_study()

In [26]:
from optuna.samplers import TPESampler
import optuna

sampler = TPESampler(seed = 0)

def objective(trial):
    params = {
        'objective': 'regression',
        'verbose': -1,
        'max_depth': trial.suggest_int('max_depth',5, 20),
        'num_leaves': trial.suggest_int('num_leaves', 10, 40),
        'learning_rate': trial.suggest_float("learning_rate", 1e-5, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 500, 2500),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.4, 1)}
    
    model = LGBMRegressor(**params)
    model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_valid, y_valid)],
         verbose = 0, early_stopping_rounds = 50)
    pred = model.predict(X_valid)
    pcc = pearsonr(pred, y_valid)[0]
    return pcc

study_model = optuna.create_study(direction = 'maximize', sampler = sampler)
study_model.optimize(objective, n_trials = 20) 

[I 2022-02-11 07:49:52,374] A new study created in memory with name: no-name-42ebcc82-24fd-4b5e-9112-7747e137d53b
[I 2022-02-11 07:50:07,549] Trial 0 finished with value: 0.6147841954237596 and parameters: {'max_depth': 13, 'num_leaves': 32, 'learning_rate': 0.06028030997340368, 'n_estimators': 1590, 'min_child_samples': 45, 'subsample': 0.7875364678399936}. Best is trial 0 with value: 0.6147841954237596.
[I 2022-02-11 07:50:19,360] Trial 1 finished with value: 0.6127261737365276 and parameters: {'max_depth': 12, 'num_leaves': 37, 'learning_rate': 0.09636663942249793, 'n_estimators': 1267, 'min_child_samples': 81, 'subsample': 0.7173369518517427}. Best is trial 0 with value: 0.6147841954237596.
[I 2022-02-11 07:51:17,670] Trial 2 finished with value: 0.6157302227101291 and parameters: {'max_depth': 14, 'num_leaves': 38, 'learning_rate': 0.007112895459206715, 'n_estimators': 674, 'min_child_samples': 6, 'subsample': 0.8995719073287628}. Best is trial 2 with value: 0.6157302227101291.
[I

In [27]:
# select best trial and parameter
trial = study_model.best_trial
best_params = trial.params

print('Best params from optuna: \n', best_params)

Best params from optuna: 
 {'max_depth': 19, 'num_leaves': 16, 'learning_rate': 0.03666757949384986, 'n_estimators': 523, 'min_child_samples': 89, 'subsample': 0.4011656977680331}


#### Plots from optuna
Optuna provides visualization of result from hyperparmeter tuning.

1. plot_optimization_history
  + plot optimization history of all trials in a study
<br>
2. plot_slice
  + plot the parameter relationship as slice plot in a study.
<br>  
3. plot_param_importances
  + plot hyperparameter importances 

In [28]:
optuna.visualization.plot_optimization_history(study_model)

In [29]:
optuna.visualization.plot_slice(study_model)

In [30]:
optuna.visualization.plot_param_importances(study_model)

### (3) Prediction with selected optimal parameter

In [31]:
opt_model = LGBMRegressor(**best_params)

cv_result(opt_model, X, y)

10it [04:13, 25.31s/it]

0th fold: LGBMRegressor PCC: 0.9927
1th fold: LGBMRegressor PCC: 0.9893
2th fold: LGBMRegressor PCC: 0.9881
3th fold: LGBMRegressor PCC: 0.9704
4th fold: LGBMRegressor PCC: 0.8965
5th fold: LGBMRegressor PCC: 0.9722
6th fold: LGBMRegressor PCC: 0.9800
7th fold: LGBMRegressor PCC: 0.9808
8th fold: LGBMRegressor PCC: 0.9398
9th fold: LGBMRegressor PCC: 0.9824

LGBMRegressor average PCC: 0.9692
